In [2]:
from fastapi import FastAPI, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from src.ocr import extract_titles
from src.metadata import fetch_metadata
from src.embeddings import generate_embedding
from src.recommender import recommend_books
from src.utils import save_file
import os
import easyocr

C:\Users\Samar\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image_path = "C:/Users/Samar/OneDrive/Documents/Coding practice/ShelfScanner/shelfscanner/backend/data/shelf_images/a2b1928c-1df5-489d-a4ac-64fcbca1ca83_1000026200.jpg"
reader = easyocr.Reader(['en'])
result = reader.readtext(image_path, detail=0)
print(result)
# Simple heuristic: join all text as potential titles, split by lines or common separators
# titles = ' '.join(result).split('\n')  # Basic splitting; improve with vision if needed
# print([title.strip() for title in titles if title.strip()])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
C:\Users\Samar\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[':', '3', '2', 'I', 'AR', '8', '0', '6', '1', '3', '8', '1', '3', 'N A t ! 0 N $', 'NL', '<', 'Qu', '8', '4', '1', '1', 'A DA M', '0', 'M', '08', '0', 'S MI T H', 'OHapaTi', 'BHARAN', 'introduction By ALAN', 'B .', 'KROEGER', 'IBHAWAN', '(P', 'LD', 'eal', 'pnay eove', 'ELIZABETH GILBERT', 'LEGAL EAGLES', 'BHAX', 'H', 'Paulo Coelho', 'TlPortoblloof', '4', '2', '4', 'GIVE TAKE', '28', '1', '@2g8', '3', 'JACI HIGGINS', 'BAD GONIPAMY', 'J]', '0 &', '%', '83', '60', '40', 'The', 'Psychologyof Money', 'MORGAN HOUSEL', '88', 'THOMAS HARDY', 'umpty', 'Publications International Ltd', 'Selected Works', 'THE', 'Biography', 'TINY TOT', 'VKRAM VEIAVALL', 'RAM JETHMALANI', 'h', '2E', 'Our Environment', '48 Book Paradise(', 'REBEL', 'SUSAN ADELMAN', 'RAAAARGHI', 'JEREMY_STRONG', 'DINOSAUR', 'Pox', 'NANI PALKHIVALA', 'Soli J Sorabjee', 'BANDAGESI', 'JEREMY STRONG LET $', 'Do', 'THE', 'Pharaohi', 'GENIUS', 'Arvind P Datar', 'LexisNexis"', '6', 'COURTROOM', '0', 'NEW', 'dhisndzoDtsmixi', '9', 'DotoL',

In [4]:
titles = ' '.join(result).split('\n')
# print(titles)
print([title.strip() for title in titles if title.strip()])

[': 3 2 I AR 8 0 6 1 3 8 1 3 N A t ! 0 N $ NL < Qu 8 4 1 1 A DA M 0 M 08 0 S MI T H OHapaTi BHARAN introduction By ALAN B . KROEGER IBHAWAN (P LD eal pnay eove ELIZABETH GILBERT LEGAL EAGLES BHAX H Paulo Coelho TlPortoblloof 4 2 4 GIVE TAKE 28 1 @2g8 3 JACI HIGGINS BAD GONIPAMY J] 0 & % 83 60 40 The Psychologyof Money MORGAN HOUSEL 88 THOMAS HARDY umpty Publications International Ltd Selected Works THE Biography TINY TOT VKRAM VEIAVALL RAM JETHMALANI h 2E Our Environment 48 Book Paradise( REBEL SUSAN ADELMAN RAAAARGHI JEREMY_STRONG DINOSAUR Pox NANI PALKHIVALA Soli J Sorabjee BANDAGESI JEREMY STRONG LET $ Do THE Pharaohi GENIUS Arvind P Datar LexisNexis" 6 COURTROOM 0 NEW dhisndzoDtsmixi 9 DotoL IO RLd WIDE THP ONESTSEUEES WO RLD WHDE 1 2o 9 ROBERT 0 5 46 3 @2 3 V BESTSELLER JoHN DESTSELLER 1 1 LUDIM 1 g3 1 1 5 46 BLHILy GrUgham  BlMEY E VHHH 2 I0 3 31 3 4 3 0 6 2 1 CLEAREPRESEM CHAHEber TESUMF 1 IH APOCHPSE E1 3 HommanLeuis 8 H 1 DA_BER THE FEARG 1 1 IH L WATCH 1 3 I3/ 23 RAINMaker IV

In [5]:
import cv2
import numpy as np
import pytesseract
from paddleocr import PaddleOCR
import boto3
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from google.cloud import vision

In [6]:
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [7]:
alpha = 1.5  # Contrast control (1.0-3.0)
beta = 50    # Brightness control (0-100)
contrasted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

In [8]:
_, binary = cv2.threshold(contrasted, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [9]:
denoised = cv2.GaussianBlur(binary, (5, 5), 0)

In [10]:
scale_percent = 150  # Increase by 50%
width = int(denoised.shape[1] * scale_percent / 100)
height = int(denoised.shape[0] * scale_percent / 100)
resized = cv2.resize(denoised, (width, height), interpolation=cv2.INTER_CUBIC)

In [11]:
result = reader.readtext(resized, detail=0)
# print(result)
# Simple heuristic: join all text as potential titles, split by lines or common separators
titles = ' '.join(result).split('\n')  # Basic splitting; improve with vision if needed
print([title.strip() for title in titles if title.strip()])

C:\Users\Samar\AppData\Roaming\Python\Python313\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


['# 7 0 4 A 2 f 1 AS S M ] T % 00 itroducfiow %" Gii7 2 pcy loxe ELIzABETh GILBERT: 1 ` L- BHAN Paulo Coelho Tortobuloet 33 YE64d z 0 JACU ES 3 0 _ ~0 2 Du 107\' Mukuanhuusel 33aS HARDY THE IOT VOYIWZ MALANI 8 F REB DELMAN WEnOSAUR Pow NAN Sol Sora LANdAGES\' ZEREY NG LEt*$ DO THE phara@hl COURTROOM GENIUS Arvind P Datar LexisNexis" 6 NEW bindzodi_au 6 @ @ ui WOBLOMIde 1 ROBERT 0 D 0 3 12 Mau 1 1 2 LudLuM 8 F B VLUUDA MMu U 1 2 401 1 9 2 ~ 1 ELvB i; CHAHEBER Tm a1 Kzeh APOCHESE 1 7 Norman Lcrzs 0 DHncR RanhEker #4 [ WATCA 1 1 33 JF De GOL SaB 0 3 ECTCE TZETE 413 Turnip 1 1 0 SHl 9 3 1 1 0 1']


In [12]:
#pytesseract
# Configure Tesseract path (update if necessary)
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"  # Windows example

# Load and preprocess image
image = cv2.imread(image_path, 0)  # Grayscale
_, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarization

# Extract text
text = pytesseract.image_to_string(binary, lang='eng', config='--psm 6')  # PSM 6 for single block of text
print("Tesseract Output:", text)

Tesseract Output: B | . a MEY a 2 | TE a Mela i a AT & OG N S A ol Bee CS | je
‘(foo || SBA PACT [Ea
is ww Be Ue (¥ Ne il se ms « TH - 2a (oe | Oo
¥ g Sls ' | | +, 4 If; ; l : G- ( : i! H ect i graaten BY ALAN B. RRGES Bi. By —— EME — = ah aceite
“s a eB es i S. | i: am Gh: : nd Hee | fae j i Etter POM a & penn
: ee S .  FAGLES BHAN = p23. *
be Paulo Corlho Poo a ae MIN *e58°3 ad _.. BEE Se THOMAS HARDY (a
. € in = - " oe THE Bathe aati. att Bingrapi: ze ve _ : Se ® = a -
/ mvtor——_VIRRA VEAL @ i | & REB E BEC SABRADELMAN | * ————————=ESSSSS
a. i) CULE NGG tne oe ayer S _ 4 SS es @} (8) a 5 = 6
Se Seely] nn nc . Soli ) Sorabjee LexisNexis’ | I" = = e as a
Bg SY SONG DENOSAUR POX opm . NANI TRB GERIUS Arvind P Datar a, a en | » 6s ie. [>
waladincadmtar 7 EREMYTSYRQNG LET'S D0 THE PHARAOH! 6 Co —— wet 4 mE Bye | ‘BORERT me! ame aah pore
fe us aa. ae { A SOHN | seathce =: Bees: cole eae © ie
2) be ofc ze | = | % BESTSELLER GRISHAM: Tim " ie 4 Fy Fs ist ge eRe | 2S gS Sm
a (YE : » os > Tom

In [13]:
# Load and preprocess image
image = cv2.imread(image_path, 0)  # Grayscale
_, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Enable text orientation detection


C:\Users\Samar\AppData\Local\Temp\ipykernel_42804\3129496847.py:6: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Enable text orientation detection
C:\Users\Samar\AppData\Roaming\Python\Python313\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\Samar\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\U

In [ ]:
# Extract text
# result = ocr.predict(binary)
result = ocr.predict(image_path)
# print(result)
text = ' '.join([line[1][0] for line in result[0]])  # Extract text from results
print("PaddleOCR Output:", text)

Resized image size (3072x4096) exceeds max_side_limit of 4000. Resizing to fit within limit.


[{'input_path': 'C:/Users/Samar/OneDrive/Documents/Coding practice/ShelfScanner/shelfscanner/backend/data/shelf_images/a2b1928c-1df5-489d-a4ac-64fcbca1ca83_1000026200.jpg', 'page_index': None, 'doc_preprocessor_res': {'input_path': None, 'page_index': None, 'input_img': array([[[207, ..., 218],
        ...,
        [ 57, ..., 222]],

       ...,

       [[109, ..., 196],
        ...,
        [ 29, ..., 106]]], shape=(3072, 4096, 3), dtype=uint8), 'model_settings': {'use_doc_orientation_classify': True, 'use_doc_unwarping': True}, 'angle': 0, 'rot_img': array([[[207, ..., 218],
        ...,
        [ 57, ..., 222]],

       ...,

       [[109, ..., 196],
        ...,
        [ 29, ..., 106]]], shape=(3072, 4096, 3), dtype=uint8), 'output_img': array([[[170, ..., 213],
        ...,
        [112, ..., 155]],

       ...,

       [[ 27, ..., 140],
        ...,
        [ 11, ...,  48]]], shape=(3072, 4096, 3), dtype=uint8)}, 'dt_polys': [array([[346,   5],
       ...,
       [368, 256]], sh

In [15]:
from google.oauth2 import service_account

In [16]:
# credentials = service_account.Credentials.from_service_account_file(
#     "C:\\Users\\Samar\\OneDrive\\Documents\\Coding practice\\ShelfScanner\\promptopia-440612-83a68e2fab08.json"  # Use double backslashes on Windows
# )

credentials = service_account.Credentials.from_service_account_file(
    "C:/Users/Samar/OneDrive/Documents/Coding practice/ShelfScanner/promptopia-440612-83a68e2fab08.json"  # Use double backslashes on Windows
)

client = vision.ImageAnnotatorClient(credentials=credentials)
with open(image_path, 'rb') as image_file:
    content = image_file.read()
image = vision.Image(content=content)
response = client.text_detection(image=image)
if response.text_annotations:
    print("Extracted Text:", response.text_annotations[0].description)
else:
    print("No text found.")
if response.error.message:
    raise Exception(response.error.message)

Extracted Text: t 2012-2013
020132014
VOLUME: 14
LO Volume: 15
UHAMMIKA & SUSAN HARMER
#2
Issue 25 to 28
Panini BOOKS
Panini BOOKS
VMS
2015
TIMES
FT
E TALES
THE PIED PIPER OF HAMELIN
ADA
SMAT
NATIONS
ADAM
SMITH
त्र मस्ती भरी कहानियाँ
BAL STORIES
and Computer Design
जल बुक
HC Verma
HC Verma
Vasan
BHARATI
BHARATI
BHAWAN
BHAWAN
(P&D)
INTRODUCTION BY ALAN B. KRUEGER
PORTFOLIO
PENGUIN
Turnip
The Enormous Turnip
Ladv
AR
The ARABIAN NIGHTS
IN
GRAMERCY
JACK HIGGINS BAD COMPANY
THOMAS HARDY
Selected Works
ARCHER
JEFFREY
U.S.
$7.50
CAN
$9.50
PLANS
THE BEST LAID
SHINE DOWN
THE STAR
Grafton
PENGUIN
JAICO
SHOBHAA
BOOK
DE
INDU
LEGAL EAGLES BHAN
GRANT
ADAM
GIVE TAKE
рипа
The Psychology of Money
HELPING
WHY
OTHERS
DRIVES
SUCCESS
OUR
MORGAN HOUSEL
NOM
A
eat pray love ELIZABETH GILBERT
Paulo Coelho
The Witch of
Portobello
Humpty Dumpty
Publications International, Ltd.
VIKRAM VETAAL
HARPER
LexisNexis
6
CITY OF
IDERS SHADOWS
TIME
ORIDERS ROME
GATES OF
TIME
CORIDERS
ALEX
SCARROW
ALEX
SCARROW
ALEX
SCARROW
T